In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8faa100


In [2]:
n = 3

x = []
for i in range(n):
    x.append(ROOT.RooRealVar(f"x_{i}", f"x_{i}", 0., -500000., 500000.))

In [3]:
# C = ROOT.std.vector([3941018.25888312, 908254.1753605, -1930641.5492786, 908254.1753605, 3009327.845014, 486358.59468932, -1930641.5492786, 486358.59468932, 7170290.07207485])
# cov_mat = ROOT.TMatrixD(n, n, C.data())
cov_mat = ROOT.TMatrixD(ROOT.TMatrixD.kUnit, ROOT.TMatrixD(n, n))
# print(cov_mat[0][0], cov_mat[0][1])
gen_model = ROOT.RooMultiVarGaussian("gauss", "gauss", ROOT.RooArgList(x), cov_mat)

In [4]:
p = []
y = []
squares = []
for i in range(n):
    p.append(ROOT.RooRealVar(f"p_{i}", f"p_{i}", 0., -500, 500))
    y.append(ROOT.RooAddition(f"y_{i}", f"y_{i}", x[i], p[i]))
    squares.append(ROOT.RooProduct(f"square_{i}", f"square_{i}", y[i], y[i]))

In [5]:
chi2 = ROOT.RooAddition("chi2_calc", "chi2_calc", ROOT.RooArgList(squares))

In [6]:
chi2_ndf = ROOT.RooConstVar("ndf", "ndf", n)
fit_model = ROOT.RooChiSquarePdf("chi2_pdf", "chi2_pdf", chi2, chi2_ndf)

In [7]:
fit_model2 = ROOT.RooMultiVarGaussian("gauss2", "gauss2", x, p, cov_mat)

In [8]:
data = gen_model.generate(x, 1)
# data = gen_model.generate(x, 10)
# data = gen_model.generate(x, 100)
# data2 = gen_model.generate(x, 1)
# data = gen_model.generate(x, 100000)
data.Print()

RooDataSet::gaussData[x_0,x_1,x_2] = 1 entries


In [9]:
# abuse ConditionalObservables to inhibit integration
# nll = fit_model.createNLL(data, EvalBackend=("cpu"), ConditionalObservables=x)
nll2 = fit_model2.createNLL(data, EvalBackend=("cpu"), ConditionalObservables=x)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(gauss2) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 2.08121 ms


In [10]:
# nll.Print("t")

In [11]:
%time
# ROOT.RooMinimizer(nll).migrad()
ROOT.RooMinimizer(nll2).migrad()

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 4.29 μs


0

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_gauss2_gaussData) Summation contains a RooNLLVar, using its error level
Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 2.6678659281746071e-13
Edm   = 2.6675496119446028e-13
Nfcn  = 42
p_0	  = 0.998932	 +/-  0.999999	(limited)
p_1	  = -0.434764	 +/-  0.999999	(limited)
p_2	  = 0.781796	 +/-  0.999999	(limited)


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 127.642 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      0.8990460174 Edm =      0.8990460174 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 0.8990460174
  Edm           : 0.8990460174
  Internal parameters:	[                0                0                0]	
  Internal gradient  :	[     -499.4663554      217.3821932     -390.8981255]	
  Internal covariance matrix:
[[  8.0000001e-06              0              0]
 [              0  8.0000001e-06              0]
 [              0              0  8.0000001e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =      0.8990460174 Edm =      0.8990460174 NCalls =     13
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =   2.6

In [12]:
# pll0 = nll.createProfile({x[0]})
pll0 = nll2.createProfile({p[0]})

In [13]:
frame = p[0].frame(Range=(-15,15))
# nll.plotOn(frame, ShiftToZero=True)
nll2.plotOn(frame, ShiftToZero=True)
# nll2.plotOn(frame, LineColor="r")
pll0.plotOn(frame, LineColor="r")

[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[p_0]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_gauss2_gaussData) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[p_0]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[p_0]) minimum found at (p_0=0.998933)
..........................................................................................................................................................................................................

In [14]:
c = ROOT.TCanvas()
frame.Draw()
c.Draw()